In [1]:
#@title Installing needed dependencies!
from IPython.display import clear_output
try:
    import flair
    from flair.data import Sentence
    from flair.models import SequenceTagger
    from tqdm import tqdm
    import os
    import sys
    import pandas as pd
    import re
    import emoji
    print("Modules required are already installed!")
except ModuleNotFoundError:
    %pip install flair
    clear_output()
    %pip install tqdm
    clear_output()
    %pip install pandas
    clear_output()
    %pip install emoji
    print("All modules required were sucessfully installed!")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.9/240.9 kB 14.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for emoji: filename=emoji-2.2.0-py3-none-any.whl size=234926 sha256=d5ee59ca832fb18587189ba6b7e41afd54373828ba19d38ea56c3eb4a08bbd2e
  Stored in directory: /root/.cache/pip/wheels/02/3d/88/51a592b9ad17e7899126563698b4e3961983ebe85747228ba6
Successfully built emoji
All modules required were sucessfully installed!


In [2]:
#@title Load model
import re
import emoji
import flair
from flair.data import Sentence
from flair.models import SequenceTagger
from tqdm import tqdm
import os
import sys
import pandas as pd
import pandas as pd
from collections import Counter
import nltk
nltk.download('punkt')



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
#@title App English Reviews with NER [(model here)](https://huggingface.co/flair/ner-english-large)

# Initialize Flair Sentiment Classifier
tagger = SequenceTagger.load("flair/ner-english-large")
#@markdown * ### Pick store:
store = "Google Play Store" #@param ["App Store", "Google Play Store"]
#@markdown * ### Pick app: 
app = "mimo" #@param ["codeacademy", "datacamp", "encode", "learn-python-programiz", "mimo", "programming-hero", "programming-hub", "python-x", "sololearn"]

dir = "https://raw.githubusercontent.com/lopezbec/AI_Gamification_Python/main/reviews/review-files/"

if store == "App Store":
    input_file = "applestore-review-" + app + ".csv"
else:
    input_file = "googleplay-review-" + app + ".csv"

# Load the CSV file into a dataframe
df = pd.read_csv(dir + input_file)

output_df = pd.DataFrame(columns=['reviewId', 'NER_Text', 'Start_Pos', 'Eng_Pos', 'NER_Label', 'Prob'])

# iterate over each row of the input dataframe
for index, row in tqdm( df.iterrows()):
    # get the content and content_ID from the current row
    if store == "App Store":
      content = row['review']
    else:
      content = row['content']
    content_id = row[0]
    try: 
        # Convert emojis into words
        content = emoji.demojize(content, delimiters=(' ', ' '))
        # Remove all punctuation
        content = re.sub(r'[^\w\s]|_', '', content)
        # Make the text lowercase
        content = content.lower()
    # create a Flair sentence from the content
        sentence = Sentence(content)
   # print(sentence)
    # run NER on the sentence
        tagger.predict(sentence)
    except:
        continue
    
    # iterate over each entity in the sentence and add it to the output dataframe
    for entity in sentence.get_spans('ner'):
        
        new_row = pd.Series({
            'reviewId': content_id,
            'NER_Text': entity.text,
            'Start_Pos': entity.start_position,
            'End_Pos': entity.end_position,
            'NER_Label': entity.tag,
            'Prob': entity.score})

        output_df = pd.concat([output_df, new_row.to_frame().T], ignore_index=True)

#@markdown * ### Display data?
visualize = "Yes" #@param ["Yes", "No"]
if visualize == "Yes":
    display(output_df)
    # Group the dataframe by NER_Label
    groups = output_df.groupby('NER_Label')

    # Initialize a list to store the results
    results = []

    # Loop through each group
    for name, group in groups:
        # Concatenate all the NER_Text strings into a single long string
        text = ' '.join(group['NER_Text'].tolist())
        
        # Tokenize the string into words
        words = nltk.word_tokenize(text)
        
        # Count the frequency of each word
        counts = Counter(words)
        
        # Sort the words by frequency and select the top 5
        top_words = counts.most_common(5)
        
        # Add the results to the list
        for word, count in top_words:
            results.append({'NER_Label': name, 'Word': word, 'Count': count})

    # Convert the results to a dataframe
    results_df = pd.DataFrame(results)

    # Print the results dataframe
    print(results_df)
else:
    print("No display")
#@markdown * ### Save to csv?
csv = "Yes" #@param ["Yes", "No"]
if csv == "Yes":
    # Get the output file name by appending "_sentiment" before the file extension
    output_file = os.path.splitext(input_file)[0] + "_NER.csv"
    # Save the new CSV file with sentiment column
    output_df.to_csv(output_file, index=False)
else:
    print("No export")

2023-05-09 13:24:45,333 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>


637it [00:21, 38.83it/s]

2023-05-09 13:25:08,891 Warning: An empty Sentence was created! Are there empty strings in your dataset?


18012it [09:22, 34.47it/s]

2023-05-09 13:34:10,146 Warning: An empty Sentence was created! Are there empty strings in your dataset?


...

82568it [42:42, 34.64it/s]

2023-05-09 14:07:30,488 Warning: An empty Sentence was created! Are there empty strings in your dataset?


86724it [44:58, 32.14it/s]


,reviewId,NER_Text,Start_Pos,Eng_Pos,NER_Label,Prob,End_Pos
0,52,mimo,0,NaN,LOC,0.422095,4
1,76,mimos,60,NaN,PER,0.882953,65
2,145,programing hub,88,NaN,MISC,0.738259,102
3,205,pakistan,10,NaN,LOC,0.999898,18
4,234,mimo,434,NaN,ORG,0.993159,438
...,...,...,...,...,...,...,...
1803,86463,duolingo,30,NaN,MISC,0.860987,38
1804,86564,aewsome,0,NaN,ORG,0.967595,7
1805,86572,udemy,41,NaN,ORG,0.951251,46
1806,86606,duolingo,184,NaN,MISC,0.572173,192


   NER_Label        Word  Count
0        LOC         nyc     37
1        LOC       india     30
2        LOC        mimo     27
3        LOC  bangladesh     15
4        LOC        iran      7
5       MISC    duolingo    196
6       MISC        mimo    112
7       MISC     english     30
8       MISC      arabic     21
9       MISC     spanish     12
10       ORG        mimo    262
11       ORG      google     65
12       ORG    redheart     26
13       ORG        gmbh     14
14       ORG       udemy     13
15       PER        mimo    146
16       PER    redheart     18
17       PER        mimi     11
18       PER         god      7
19       PER      okhand      7


In [4]:
#@title App Spanish Reviews with NER [(model here)](https://huggingface.co/flair/ner-spanish-large)
tagger = SequenceTagger.load("flair/ner-spanish-large")
#@markdown * ### Pick store:
#@markdown Only Google Play Store Reviews were capture in Spanish due to the limitation of the API
#@markdown * ### Pick app: 
app = "mimo" #@param ["codeacademy", "datacamp", "learn-python-programiz", "mimo", "programming-hero", "programming-hub", "sololearn"]

dir = "https://raw.githubusercontent.com/lopezbec/AI_Gamification_Python/main/reviews/review-files-Spanish/"

input_file = app + "_Spanish.csv"

# Load the CSV file into a dataframe
df = pd.read_csv(dir + input_file)

output_df = pd.DataFrame(columns=['reviewId', 'NER_Text', 'Start_Pos', 'End_Pos', 'NER_Label', 'Prob'])

# iterate over each row of the input dataframe
for index, row in tqdm( df.iterrows()):
    # get the content and content_ID from the current row
    content = row['content']
    content_id = row[0]
    try: 
        # Convert emojis into words
        content = emoji.demojize(content, delimiters=(' ', ' '))
        # Remove all punctuation
        content = re.sub(r'[^\w\s]|_', '', content)
        # Make the text lowercase
        content = content.lower()
    # create a Flair sentence from the content
        sentence = Sentence(content)
        
    # run NER on the sentence
        tagger.predict(sentence)
        #print(tagger.predict(sentence))
    except:
        continue
    
    # iterate over each entity in the sentence and add it to the output dataframe
    for entity in sentence.get_spans('ner'):
        
        new_row = pd.Series({
            'reviewId': content_id,
            'NER_Text': entity.text,
            'Start_Pos': entity.start_position,
            'End_Pos': entity.end_position,
            'NER_Label': entity.tag,
            'Prob': entity.score})

        output_df = pd.concat([output_df, new_row.to_frame().T], ignore_index=True)

#@markdown * ### Display data?
visualize = "Yes" #@param ["Yes", "No"]
if visualize == "Yes":
    display(output_df)
    # Group the dataframe by NER_Label
    groups = output_df.groupby('NER_Label')

    # Initialize a list to store the results
    results = []

    # Loop through each group
    for name, group in groups:
        # Concatenate all the NER_Text strings into a single long string
        text = ' '.join(group['NER_Text'].tolist())
        
        # Tokenize the string into words
        words = nltk.word_tokenize(text)
        
        # Count the frequency of each word
        counts = Counter(words)
        
        # Sort the words by frequency and select the top 5
        top_words = counts.most_common(5)
        
        # Add the results to the list
        for word, count in top_words:
            results.append({'NER_Label': name, 'Word': word, 'Count': count})

    # Convert the results to a dataframe
    results_df = pd.DataFrame(results)

    # Print the results dataframe
    print(results_df)
else:
    print("No display")
#@markdown * ### Save to csv?
csv = "Yes" #@param ["Yes", "No"]
if csv == "Yes":
    # Get the output file name by appending "_sentiment" before the file extension
    output_file = os.path.splitext(input_file)[0] + "_NER.csv"
    # Save the new CSV file with sentiment column
    output_df.to_csv(output_file, index=False)
else:
    print("No export")

2023-05-09 14:10:23,418 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-LOC, S-ORG, B-PER, I-PER, E-PER, S-MISC, B-ORG, E-ORG, S-PER, I-ORG, B-LOC, E-LOC, B-MISC, E-MISC, I-MISC, I-LOC, <START>, <STOP>


6901it [03:39, 37.83it/s]

2023-05-09 14:14:04,626 Warning: An empty Sentence was created! Are there empty strings in your dataset?


9253it [04:52, 31.59it/s]


,reviewId,NER_Text,Start_Pos,End_Pos,NER_Label,Prob
0,18,premio,180,186,MISC,0.788097
1,19,c edit,150,156,MISC,0.8304
2,36,cuba,127,131,LOC,0.815853
3,36,pensiveface,133,144,MISC,0.863801
4,62,grinningfacewithbigeyes,50,73,MISC,0.582291
...,...,...,...,...,...,...
276,12739,phyton,24,30,MISC,0.889052
277,12739,phyton,339,345,MISC,0.908119
278,12739,amazon,349,355,ORG,0.820831
279,12786,co te ido,3,12,MISC,0.999814


   NER_Label                     Word  Count
0        LOC                     cuba      5
1        LOC                  chévere      3
2        LOC                 colombia      2
3        LOC                    latam      2
4        LOC                 duolingo      2
5       MISC                   okhand     12
6       MISC  grinningfacewithbigeyes     11
7       MISC                sololearn      8
8       MISC                 duolingo      8
9       MISC                     play      8
10       ORG                 duolingo     20
11       ORG                    unity      4
12       ORG                     pvta      2
13       ORG                    udemy      2
14       ORG                   phyton      1
15       PER                 buenardo      6
16       PER                 duolingo      3
17       PER                 redheart      2
18       PER                     atte      1
19       PER                     juan      1
